In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

# Create list of tours

## Highest-grossing tours 
https://en.wikipedia.org/wiki/List_of_highest-grossing_concert_tours

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_concert_tours"

In [3]:
response = requests.get(
	url=url,
)
print(response.status_code)

200


In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
captions = [
    "Top 20 highest-grossing tours of all time",
    "Top 10 highest-grossing tours of the 1980s",
    "Top 10 highest-grossing tours of the 1990s",
    "Top 10 highest-grossing tours of the 2000s",
    "Top 10 highest-grossing tours of the 2010s",
    "Top 10 highest-grossing tours of the 2020s"
]

In [121]:
URL = "https://en.wikipedia.org/wiki/List_of_highest-grossing_concert_tours"
def create_tour_list(captions, url):
    df = pd.DataFrame()
    for caption in captions:
        df_tmp = pd.read_html(url, 
                          match=caption)[0]
        
        tour_links = pd.read_html(url, 
                  match=caption,
                  extract_links="all")[0][("Tour title", None)]
        
        tour_links = pd.DataFrame(tour_links.tolist(), columns=["Tour title", "Tour link"])
        df_tmp = df_tmp.merge(tour_links, on="Tour title")

        df = pd.concat([df, df_tmp]).drop_duplicates('Tour title').reset_index(drop=True)
        print(df.shape)
    
    return df

df = create_tour_list(captions, url)

(20, 11)
(30, 12)
(40, 12)
(45, 12)
(45, 12)
(49, 12)


In [123]:
df = df.rename(columns={df.columns[3]: 'Adjusted gross (in 2022 dollars)',
                   df.columns[-1]: 'Adjusted gross (in 2022 dollar)',
                   df.columns[2]: 'Actual gross'})

In [124]:
df.loc[df['Adjusted gross (in 2022 dollars)'].isna(), 'Adjusted gross (in 2022 dollars)'] = df['Adjusted gross (in 2022 dollar)']

In [126]:
df = df[["Actual gross", "Adjusted gross (in 2022 dollars)", "Artist", "Tour title", "Tour link", "Year(s)", "Shows"]]

In [127]:
df

,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Tour link,Year(s),Shows
0,"$939,100,000","$939,100,000",Elton John,Farewell Yellow Brick Road,/wiki/Farewell_Yellow_Brick_Road,2018–2023,330
1,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,/wiki/The_Eras_Tour,2023,56
2,"$776,200,000","$888,442,379",Ed Sheeran,÷ Tour,/wiki/%C3%B7_Tour,2017–2019,255
3,"$736,421,586","$958,001,690",U2,U2 360° Tour,/wiki/U2_360%C2%B0_Tour,2009–2011,110
4,"$667,726,905","$667,726,905",Coldplay,Music of the Spheres World Tour †,/wiki/Music_of_the_Spheres_World_Tour,2022–2023,114
5,"$617,300,000","$617,800,000",Harry Styles,Love On Tour,/wiki/Love_On_Tour,2021–2023,169
6,"$584,200,000","$668,678,225",Guns N' Roses,Not in This Lifetime... Tour,/wiki/Not_in_This_Lifetime..._Tour,2016–2019,158
7,"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,/wiki/Renaissance_World_Tour,2023,56
8,"$558,255,524","$787,883,017",The Rolling Stones,A Bigger Bang Tour,/wiki/A_Bigger_Bang_Tour,2005–2007,144
9,"$546,500,000","$590,190,470",The Rolling Stones,No Filter Tour,/wiki/No_Filter_Tour,2017–2021,58


## List of most-attended concert tours
https://en.wikipedia.org/wiki/List_of_most-attended_concert_tours

# Gather data about tours